In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [4]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
df_fake["class"] = 0
df_true["class"] = 1

In [7]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [20]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_true.drop(df_true.tail(10).index, inplace=True)
    
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_fake.drop(df_fake.tail(10).index, inplace=True)

In [21]:
df_fake.shape, df_true.shape

((23271, 5), (21207, 5))

In [22]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

/var/folders/4k/rfvcwwg91kzcrk4f47c087_h0000gn/T/ipykernel_67232/860779283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
/var/folders/4k/rfvcwwg91kzcrk4f47c087_h0000gn/T/ipykernel_67232/860779283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [23]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23361,BOILER ROOM – EP #55 – Roasting the Wretched H...,Tune in to the Alternate Current Radio Network...,Middle-east,"May 4, 2016",0
23362,Cruz & Kasich Quit: Trump Crushes Elite Establ...,21st Century Wire says The field is now clear ...,Middle-east,"May 4, 2016",0
23363,False Flag Florida: FBI Agents ‘Posing As Terr...,21st Century Wire says The FBI is walking a ve...,Middle-east,"May 3, 2016",0
23364,Shock to The System: New Poll Says Trump CAN B...,21st Century Wire says This is the news that t...,Middle-east,"May 3, 2016",0
23365,BOILER ROOM – EP #53 – Say Bye Bye to Culture,Tune in to the Alternate Current Radio Network...,Middle-east,"May 1, 2016",0
23366,BREAKING BAD: John McCain’s Campaign Rocked by...,21st Century Wire asks Will this be the beginn...,Middle-east,"April 28, 2016",0
23367,BOILER ROOM – EP #54 – America… The End is Nigh,Tune in to the Alternate Current Radio Network...,Middle-east,"April 28, 2016",0
23368,Trump’s Foreign Policy: Promote Stability NOT ...,21st Century Wire says We are sure that the mi...,Middle-east,"April 27, 2016",0
23369,Grasping At Straws (The Illusion of Choice),Art by Dan Lacey via Etsy (See his work here)...,Middle-east,"April 22, 2016",0
23370,Hillary Clinton: Neocon War-Hawk in Waiting,Robert Parry Consortium NewsIf there were any...,Middle-east,"April 17, 2016",0


In [24]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
21197,Colombia's FARC rebels keep famous acronym for...,BOGOTA (Reuters) - Colombia s disarmed FARC re...,worldnews,"August 31, 2017",1
21198,"Jails, justice system at breaking point as Phi...",MANILA Reuters) - In a teeming prison for unde...,worldnews,"August 31, 2017",1
21199,Panama ex-president facing political spying ch...,(Reuters) - A federal judge in Miami on Thursd...,worldnews,"August 31, 2017",1
21200,Iraqi prime minister declares victory over IS ...,"ERBIL, Iraq/BAGHDAD (Reuters) - Iraqi Prime Mi...",worldnews,"August 31, 2017",1
21201,Brazil anti-graft head defends graft fines aft...,BRASILIA (Reuters) - A top Brazilian federal p...,worldnews,"August 31, 2017",1
21202,U.S. bombers drill over Korean peninsula after...,SEOUL/TOKYO (Reuters) - South Korean and Japan...,worldnews,"August 29, 2017",1
21203,Chilean economic officials resign in blow to c...,SANTIAGO (Reuters) - Chilean President Michell...,worldnews,"August 31, 2017",1
21204,"U.S. retaliates against Russia, orders closure...",WASHINGTON (Reuters) - The United States has t...,worldnews,"August 31, 2017",1
21205,Muslim pilgrims in Muzdalifa prepare for haj's...,"MUZDALIFA, Saudi Arabia (Reuters) - Two millio...",worldnews,"August 31, 2017",1
21206,French foreign minister to travel to Libya to ...,PARIS (Reuters) - France s foreign minister sa...,worldnews,"August 31, 2017",1


In [25]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [26]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [27]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [28]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [29]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [30]:
df = df.sample(frac = 1)

In [31]:
df.head()

,text,class
15414,The leftist propaganda press is finally gettin...,0
12383,The press was not happy and took to Twitter to...,0
7629,The GOP is known for its misogyny. This is esp...,0
14324,There are two different Donald Trump s Here s...,0
2022,SAN FRANCISCO (Reuters) - Affordable housing a...,1


In [32]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [33]:
df.columns

Index(['text', 'class'], dtype='object')

In [34]:
df.head()

,text,class
0,The leftist propaganda press is finally gettin...,0
1,The press was not happy and took to Twitter to...,0
2,The GOP is known for its misogyny. This is esp...,0
3,There are two different Donald Trump s Here s...,0
4,SAN FRANCISCO (Reuters) - Affordable housing a...,1


In [35]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [36]:
df["text"] = df["text"].apply(wordopt)

In [37]:
x = df["text"]
y = df["class"]

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [40]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [41]:
pred_lr=LR.predict(xv_test)

In [42]:
LR.score(xv_test, y_test)

0.9870503597122302

In [43]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5944
           1       0.98      0.99      0.99      5176

    accuracy                           0.99     11120
   macro avg       0.99      0.99      0.99     11120
weighted avg       0.99      0.99      0.99     11120



In [53]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [60]:
pred_dt = DT.predict(xv_test)

In [61]:
DT.score(xv_test, y_test)

0.9962230215827338

In [72]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5944
           1       1.00      1.00      1.00      5176

    accuracy                           1.00     11120
   macro avg       1.00      1.00      1.00     11120
weighted avg       1.00      1.00      1.00     11120



In [75]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [76]:
pred_gbc = GBC.predict(xv_test)

In [77]:
GBC.score(xv_test, y_test)

0.99568345323741

In [78]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5944
           1       0.99      1.00      1.00      5176

    accuracy                           1.00     11120
   macro avg       1.00      1.00      1.00     11120
weighted avg       1.00      1.00      1.00     11120



In [79]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [80]:
pred_rfc = RFC.predict(xv_test)

In [81]:
RFC.score(xv_test, y_test)

0.989478417266187

In [82]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5944
           1       0.99      0.99      0.99      5176

    accuracy                           0.99     11120
   macro avg       0.99      0.99      0.99     11120
weighted avg       0.99      0.99      0.99     11120



In [83]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [85]:
news = str(input())
manual_testing(news)

 NATO allies on Tuesday welcomed President Donald Trump s decision to commit more forces to Afghanistan, as part of a new U.S. strategy he said would require more troops and funding from America s partners. Having run for the White House last year on a pledge to withdraw swiftly from Afghanistan, Trump reversed course on Monday and promised a stepped-up military campaign against  Taliban insurgents, saying:  Our troops will fight to win .  U.S. officials said he had signed off on plans to send about 4,000 more U.S. troops to add to the roughly 8,400 now deployed in Afghanistan. But his speech did not define benchmarks for successfully ending the war that began with the U.S.-led invasion of Afghanistan in 2001, and which he acknowledged had required an   extraordinary sacrifice of blood and treasure .  We will ask our NATO allies and global partners to support our new strategy, with additional troops and funding increases in line with our own. We are confident they will,  Trump said. Th



LR Prediction: Not A Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Not A Fake News


In [86]:
news = str(input())
manual_testing(news)

 Vic Bishop Waking TimesOur reality is carefully constructed by powerful corporate, political and special interest sources in order to covertly sway public opinion. Blatant lies are often televised regarding terrorism, food, war, health, etc. They are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms.The practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this. The entire role of this brainwashing industry is to figure out how to spin information to journalists, similar to the lobbying of government. It is never really clear just how much truth the journalists receive because the news industry has become complacent. The messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own 90% of the media:General Electric (GE), News-Corp, Disney, Viacom, Time Warner, 



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News


In [87]:
news = str(input())
manual_testing(news)


 SAO PAULO (Reuters) - Cesar Mata Pires, the owner and co-founder of Brazilian engineering conglomerate OAS SA, one of the largest companies involved in Brazil s corruption scandal, died on Tuesday. He was 68. Mata Pires died of a heart attack while taking a morning walk in an upscale district of S o Paulo, where OAS is based, a person with direct knowledge of the matter said. Efforts to contact his family were unsuccessful. OAS declined to comment. The son of a wealthy cattle rancher in the northeastern state of Bahia, Mata Pires  links to politicians were central to the expansion of OAS, which became Brazil s No. 4 builder earlier this decade, people familiar with his career told Reuters last year. His big break came when he befriended Antonio Carlos Magalh es, a popular politician who was Bahia governor several times, and eventually married his daughter Tereza. Brazilians joked that OAS stood for  Obras Arranjadas pelo Sogro  - or  Work Arranged by the Father-In-Law.   After years o



LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
GBC Prediction: Not A Fake News 
RFC Prediction: Not A Fake News
